# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770797467632                   -0.53    9.0         
  2   -2.772141057594       -2.87       -1.31    1.0    129ms
  3   -2.772170237062       -4.53       -2.55    1.0    131ms
  4   -2.772170702618       -6.33       -3.46    2.0    200ms
  5   -2.772170721960       -7.71       -4.03    1.0    134ms
  6   -2.772170722929       -9.01       -5.00    1.0    135ms
  7   -2.772170723013      -10.08       -5.23    2.0    177ms
  8   -2.772170723015      -11.66       -6.67    1.0    141ms
  9   -2.772170723015      -13.54       -6.57    2.0    171ms
 10   -2.772170723015   +  -14.40       -6.84    1.0    163ms
 11   -2.772170723015      -13.85       -7.75    1.0    144ms
 12   -2.772170723015      -14.31       -8.80    2.0    178ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735580120435548

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770807255839                   -0.53   10.0         
  2   -2.772048516466       -2.91       -1.31    1.0    130ms
  3   -2.772082394314       -4.47       -2.63    1.0    132ms
  4   -2.772083414848       -5.99       -3.84    2.0    196ms
  5   -2.772083417749       -8.54       -4.72    2.0    183ms
  6   -2.772083417809      -10.22       -5.85    1.0    134ms
  7   -2.772083417811      -11.77       -6.41    2.0    154ms
  8   -2.772083417811      -14.45       -7.04    2.0    179ms
  9   -2.772083417811      -14.18       -7.92    1.0    148ms
 10   -2.772083417811   +  -13.94       -9.16    2.0    165ms

Polarizability via ForwardDiff:       1.772534972108326
Polarizability via finite difference: 1.7735580120435548
